In [1]:
!pip install utils

In [3]:
from scipy.stats import mannwhitneyu
import itertools
import numpy as np
from collections import defaultdict
from typing import Callable, Optional
import csv
import pandas as pd
import os
import utils

import utils


def is_valid_data_file(file_name:str) -> bool:
    return file_name.endswith("json") or file_name.endswith("txt")


def get_mean_for_combinations(df: pd.DataFrame, 
                       independent_variables: list[str], 
                       dependent_variables: list[str]) -> pd.DataFrame:

    # ensure all the columns are present in the df
    for col in independent_variables+dependent_variables:
        if col not in df:
            raise Exception(f"The column {col} is not in the dataframe\n\t(columns are {list(df.columns)})")
    assert(all(col in df for col in independent_variables))
    assert(dependent_variable in df for dependent_variable in dependent_variables)
    
    grouped = df.groupby(independent_variables, dropna=False)[dependent_variables].mean().reset_index()
    
    return grouped


import json
import os


def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Skip if 'results_by_tree' is not present
        if "results_by_tree" not in item:
            return []

        problem_name = item["problem_name"]
        pRef_method = item["pRef_method"]
        pRef_size = item["sample_size"]

        entries = item["results_by_tree"]

        def get_modified_entry(entry):
            entry["problem"] = problem_name
            entry["pRef_method"] = pRef_method
            entry["pRef_size"] = pRef_size

            errors = entry["results"]
            entry = entry | errors
            del entry["results"]

            if "order_tree" in entry:
                del entry["order_tree"]

            return entry

        entries = list(map(get_modified_entry, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]

'''
def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Handle both results_by_tree and tree_settings_list
        if "results_by_tree" in item:
            # Original logic for results data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            entries = item["results_by_tree"]
            
            def get_modified_entry(entry):
                entry["problem"] = problem_name
                entry["pRef_method"] = pRef_method
                entry["pRef_size"] = pRef_size
                errors = entry["results"]
                entry = entry | errors
                del entry["results"]
                if "order_tree" in entry:
                    del entry["order_tree"]
                return entry
            
            return list(map(get_modified_entry, entries))
            
        elif "tree_settings_list" in item:
            # New logic for configuration data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            has_error = 'error' in item
            error_msg = item.get('error', '')
            
            entries = []
            for tree_setting in item["tree_settings_list"]:
                for depth in tree_setting["depths"]:
                    entry = {
                        'problem': problem_name,
                        'pRef_method': pRef_method,
                        'pRef_size': pRef_size,
                        'kind': tree_setting['kind'],
                        'depth': depth,
                        'ps_budget': tree_setting['ps_budget'],
                        'ps_population': tree_setting['ps_population'],
                        'avoid_ancestors': tree_setting['avoid_ancestors'],
                        'metrics': tree_setting['metrics'],
                        'has_error': has_error,
                        'error_message': error_msg,
                        # No actual results available
                        'mse': None,
                        'mae': None,
                        'r_sq': None,
                        'evs': None
                    }
                    entries.append(entry)
            return entries
        else:
            return []

    return [entry for item in data for entry in item_to_list_of_entries(item)]


def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Handle both results_by_tree and tree_settings_list
        if "results_by_tree" in item:
            # Original logic for results data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            entries = item["results_by_tree"]
            
            def get_modified_entry(entry):
                entry["problem"] = problem_name
                entry["pRef_method"] = pRef_method
                entry["pRef_size"] = pRef_size
                errors = entry["results"]
                entry = entry | errors
                del entry["results"]
                if "order_tree" in entry:
                    del entry["order_tree"]
                return entry
            
            return list(map(get_modified_entry, entries))
            
        elif "tree_settings_list" in item:
            # FIXED: Handle different tree types properly
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            has_error = 'error' in item
            error_msg = item.get('error', '')
            
            entries = []
            for tree_setting in item["tree_settings_list"]:
                for depth in tree_setting["depths"]:
                    # Base entry with common fields
                    entry = {
                        'problem': problem_name,
                        'pRef_method': pRef_method,
                        'pRef_size': pRef_size,
                        'kind': tree_setting['kind'],
                        'depth': depth,
                        'has_error': has_error,
                        'error_message': error_msg,
                        # No actual results available
                        'mse': None,
                        'mae': None,
                        'r_sq': None,
                        'evs': None
                    }
                    
                    # Add tree-type-specific fields
                    if tree_setting['kind'] == 'ps':
                        entry.update({
                            'ps_budget': tree_setting.get('ps_budget', None),
                            'ps_population': tree_setting.get('ps_population', None),
                            'avoid_ancestors': tree_setting.get('avoid_ancestors', None),
                            'metrics': tree_setting.get('metrics', None),
                            'cp': None  # PS doesn't have cp
                        })
                    elif tree_setting['kind'] == 'iai':
                        entry.update({
                            'ps_budget': None,  # IAI doesn't have these
                            'ps_population': None,
                            'avoid_ancestors': None,
                            'metrics': None,
                            'cp': tree_setting.get('cp', None)
                        })
                    elif tree_setting['kind'] == 'naive':
                        entry.update({
                            'ps_budget': None,  # Naive doesn't have these
                            'ps_population': None,
                            'avoid_ancestors': None,
                            'metrics': None,
                            'cp': None
                        })
                    
                    entries.append(entry)
            return entries
        else:
            return []

    return [entry for item in data for entry in item_to_list_of_entries(item)]
'''
def convert_accuracy_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue

        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_entries(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
   
def json_to_tree_data(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Skip if 'results_by_tree' is not present
        if "results_by_tree" not in item:
            return []

        surrounding_information = {
            "problem": item["problem_name"],
            "pRef_method": item["pRef_method"]
        }

        entries = item["results_by_tree"]
        entries = [thing for thing in entries if "order_tree" in thing]

        def convert_order_tree(order_tree, accumulator=None, current_depth=0):
            if accumulator is None:
                accumulator = defaultdict(list)
            accumulator[current_depth].append(order_tree["own"])
            if len(order_tree["matching"]) > 0:
                convert_order_tree(order_tree["matching"], accumulator, current_depth + 1)
            if len(order_tree["unmatching"]) > 0:
                convert_order_tree(order_tree["unmatching"], accumulator, current_depth + 1)
            return accumulator

        def convert_tree_to_averages_by_level(entry):
            ps_search_info = {
                "ps_budget": entry["ps_budget"],
                "ps_population": entry["ps_population"],
                "metrics": entry["metrics"]
            }
            tree_structure = entry["order_tree"]
            just_depths = convert_order_tree(tree_structure)
            core_info_trees = [
                {"depth": depth, "order": order}
                for depth in just_depths
                for order in just_depths[depth]
            ]
            core_info_trees = [surrounding_information | ps_search_info | core_tree for core_tree in core_info_trees]
            return core_info_trees

        entries = list(map(convert_tree_to_averages_by_level, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]


def convert_tree_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue


        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_tree_data(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
    
    
def filter_dataframe(df, **kwargs):
    df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original
    for col, value in kwargs.items():
        if col in df.columns:
            df = df[df[col] == value]
        else:
            raise ValueError(f"Column '{col}' not found in dataframe.")
    return df
        

    
    

def prettify_kind_column(df):
    kind_dict = {#"variance":"PS-W",
                 #"variance estimated_atomicity": "PS-WA",
                 "simplicity variance": "PS-SW",
                 "simplicity variance estimated_atomicity" :"PS-SWA"}
    
    df['kind'] = df.apply(
    lambda row: (
        kind_dict[row['metrics']] if row['kind'] == 'ps' else
        'Trad.' if row['kind'] == 'naive' else
        'IAI' if row['kind'] == 'iai' else
        row['kind']
    ),
    axis=1
)
    
    

    
    

In [4]:
import os

In [6]:
#run_location = r"/Users/gian/Desktop/CondorResults/VDT/compareown/run3/"
#run_location = r"C:\Users\gac8\Desktop\CondorResults\VDT\compareown\all_final_runs"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_07-29-H15'm'15's16"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-07-H16'm'47's09"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-10-H01'm'12's39"
#A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_07-29-H15'm'15's16

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-19-H16'm'26's53"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-19-H23'm'35's39"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\iai_run_3_08-20-H01'm'00's52"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-24-H02'm'25's36"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\iai_run_3_08-24-H11'm'06's57"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-24-H02'm'25's36"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-26-H14'm'06's32"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-27-H12'm'29's47"

run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\all_final_runs"

results_csv = os.path.join(run_location, "results.csv")
tree_data_csv = os.path.join(run_location, "tree_data.csv")

convert_accuracy_data_to_df(run_location, results_csv)
convert_tree_data_to_df(run_location, tree_data_csv)

#convert_accuracy_data_to_df(os.path.join(run_location, "data"), results_csv)
#convert_tree_data_to_df(os.path.join(run_location, "data"), tree_data_csv)



In [7]:

accuracy_data = pd.read_csv(results_csv)
prettify_kind_column(accuracy_data)
#tree_data = pd.read_csv(tree_data_csv)

display(accuracy_data)
#display(accuracy_data.dtypes)
#display(tree_data)


#prettify_kind_column(tree_data)


for kind in accuracy_data["kind"].unique():
    matching_rows = accuracy_data[accuracy_data['kind'] == kind]
    print(f"For the tree kind {kind}, there are {matching_rows.shape[0]} rows")

#headers = "kind,depth,ps_budget,ps_population,avoid_ancestors,metrics,problem,pRef_method,mse,mae,r_sq,evs"

,kind,depth,ps_budget,ps_population,avoid_ancestors,metrics,problem,pRef_method,pRef_size,mse,mae,r_sq,evs,cp
0,PS-SW,3,5000.0,100.0,False,simplicity variance,GC_L,BRO,10000,232.494405,11.976377,-0.000084,-0.000077,NaN
1,PS-SWA,3,5000.0,100.0,False,simplicity variance estimated_atomicity,GC_L,BRO,10000,230.577627,12.014418,-0.000205,-0.000139,NaN
2,PS-SW,4,5000.0,100.0,False,simplicity variance,GC_L,BRO,10000,208.583458,11.401284,-0.000419,0.000161,NaN
3,PS-SWA,4,5000.0,100.0,False,simplicity variance estimated_atomicity,GC_L,BRO,10000,246.396546,12.257105,-0.019573,-0.019550,NaN
4,PS-SW,5,5000.0,100.0,False,simplicity variance,GC_L,BRO,10000,228.561854,11.957361,-0.027138,-0.027059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,PS-SWA,3,5000.0,100.0,False,simplicity variance estimated_atomicity,SAT_S,PSO,10000,12.008180,2.759144,0.133962,0.134432,NaN
296,PS-SW,4,5000.0,100.0,False,simplicity variance,SAT_S,PSO,10000,13.188697,2.894415,-0.000968,0.000000,NaN
297,PS-SWA,4,5000.0,100.0,False,simplicity variance estimated_atomicity,SAT_S,PSO,10000,9.419367,2.442990,0.324944,0.324944,NaN
298,PS-SW,5,5000.0,100.0,False,simplicity variance,SAT_S,PSO,10000,9.199014,2.406220,0.299076,0.299373,NaN


For the tree kind PS-SW, there are 45 rows
For the tree kind PS-SWA, there are 45 rows
For the tree kind IAI, there are 105 rows
For the tree kind Trad., there are 105 rows


In [8]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows


In [10]:

def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size=10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    
    # Check which methods are actually available in your data
    available_methods = usable_data["kind"].unique()
    print(f"Available methods in your data: {available_methods}")
    
    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str):
        # Get data for all available methods
        method_data = {}
        for method in available_methods:
            data = filter_dataframe(usable_data, problem=problem, depth=depth, pRef_method=metaheuristic, kind=method)[result_column]
            if len(data) > 0:  # Only include methods with data
                method_data[method] = data
        
        # If we don't have enough methods to compare, return NaN
        if len(method_data) < 2:
            return {
                "problem": problem,
                "depth": depth,
                "metaheuristic": metaheuristic,
                "comparison": "insufficient_data",
                "p_value": float('nan'),
                "method_1": None,
                "method_2": None,
                "method_1_mean": float('nan'),
                "method_2_mean": float('nan')
            }
        
        # If we have PS-SW and PS-SWA, compare them
        if "PS-SW" in method_data and "PS-SWA" in method_data:
            p_value = mannwhitneyu(method_data["PS-SW"], method_data["PS-SWA"], alternative="two-sided").pvalue
            comparison = "PS-SW_vs_PS-SWA"
            method_1, method_2 = "PS-SW", "PS-SWA"
            method_1_mean = method_data["PS-SW"].mean()
            method_2_mean = method_data["PS-SWA"].mean()
        else:
            p_value = float('nan')
            comparison = "no_valid_comparison"
            method_1, method_2 = None, None
            method_1_mean, method_2_mean = float('nan'), float('nan')
        
        return {
            "problem": problem,
            "depth": depth,
            "metaheuristic": metaheuristic,
            "comparison": comparison,
            "p_value": p_value,
            "method_1": method_1,
            "method_2": method_2,
            "method_1_mean": method_1_mean,
            "method_2_mean": method_2_mean
        }
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data



In [11]:
    
    
def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size = 10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    
    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str) -> (str, np.ndarray):
        for_each_method = {tree_method: filter_dataframe(usable_data, problem = problem, depth = depth, pRef_method = metaheuristic, kind = tree_method)[result_column]
                           for tree_method in {"PS-SW", "PS-SWA", "IAI", "Trad."}}
        
        iai_average = np.average(for_each_method["IAI"])
        naive_average = np.average(for_each_method["Trad."])
        
        
        winning_competitor_method = "IAI" if iai_average > naive_average else "Trad."
        
        p_value_between_w_and_competitor = mannwhitneyu(for_each_method["PS-SW"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        p_value_between_wa_and_competitor = mannwhitneyu(for_each_method["PS-SWA"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        
        return {"problem": problem,
                "depth": depth,
                "metaheuristic": metaheuristic,
                "p_value_sw":p_value_between_w_and_competitor,
                "p_value_swa": p_value_between_wa_and_competitor,
                "winning_competitor": winning_competitor_method}
    
    
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data["pRef_method"].unique()
    
    dicts = [winning_competitor_for_competition_and_values(problem=problem, depth = depth, metaheuristic=metaheuristic)
             for problem in all_problems
             for depth in depths
             for metaheuristic in all_metaheuristics]
    
    return pd.DataFrame(dicts)
        
        
    

statistical_data = generate_statistical_test_data(accuracy_data, None, None)

pivot_table = statistical_data.pivot_table(index=["problem", "depth", "metaheuristic"], 
                                            values =["p_value_sw", "p_value_swa"])
display(statistical_data)
display(pivot_table)

C:\Users\chand\AppData\Local\Temp\ipykernel_24148\3339840094.py:18: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  p_value_between_w_and_competitor = mannwhitneyu(for_each_method["PS-SW"], for_each_method[winning_competitor_method], alternative="greater").pvalue
C:\Users\chand\AppData\Local\Temp\ipykernel_24148\3339840094.py:19: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  p_value_between_wa_and_competitor = mannwhitneyu(for_each_method["PS-SWA"], for_each_method[winning_competitor_method], alternative="greater").pvalue


,problem,depth,metaheuristic,p_value_sw,p_value_swa,winning_competitor
0,GC_L,3,BRO,1.0,1.0,Trad.
1,GC_L,3,CRO,0.5,1.0,IAI
2,GC_L,3,PSO,1.0,1.0,IAI
3,GC_L,3,ABC,NaN,NaN,Trad.
4,GC_L,3,ACO,NaN,NaN,Trad.
5,GC_L,3,BBO,NaN,NaN,Trad.
6,GC_L,3,SSO,NaN,NaN,Trad.
7,GC_L,4,BRO,0.5,1.0,IAI
8,GC_L,4,CRO,1.0,1.0,IAI
9,GC_L,4,PSO,1.0,1.0,IAI


p_value_sw  p_value_swa
problem depth metaheuristic                         
GC_L    3     BRO                   1.0          1.0
              CRO                   0.5          1.0
              PSO                   1.0          1.0
        4     BRO                   0.5          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0
        5     BRO                   1.0          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0
GC_S    3     BRO                   1.0          1.0
              CRO                   1.0          1.0
              PSO                   1.0          0.5
        4     BRO                   1.0          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0
        5     BRO                   1.0          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0
SAT_L   3     BRO                   0.5          1.0
              CRO                   0.5          1.0
              PSO                   0.5          1.0
        4     BRO                   0.5          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0
        5     BRO                   0.5          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0
SAT_M   3     BRO                   0.5          1.0
              CRO                   0.5          1.0
              PSO                   0.5          1.0
        4     BRO                   1.0          1.0
              CRO                   0.5          1.0
              PSO                   1.0          1.0
        5     BRO                   1.0          1.0
              CRO                   0.5          1.0
              PSO                   1.0          1.0
SAT_S   3     BRO                   1.0          1.0
              CRO                   0.5          1.0
              PSO                   1.0          1.0
        4     BRO                   1.0          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0
        5     BRO                   1.0          1.0
              CRO                   1.0          1.0
              PSO                   1.0          1.0

In [7]:
statistical_data = generate_statistical_test_data(accuracy_data, None, None)


In [13]:
'''
def bold_max(row):
    row_as_numbers = [float(item[:-1]) for item in row]
    max_number = max(row_as_numbers)
    
    return ['font-weight: bold' if item == max_number else '' for item in row_as_numbers]
'''


def bold_max(row):
    row_as_numbers = []
    for item in row:
        if isinstance(item, str) and item.endswith("%"):
            # Remove % and convert string to float
            row_as_numbers.append(float(item[:-1]))
        else:
            # Already a number (int or float)
            row_as_numbers.append(float(item))
    max_number = max(row_as_numbers)
    return ['font-weight: bold' if num == max_number else '' for num in row_as_numbers]


def style_pivot_table(pivot_table):
    custom_column_order = ['PS-SW', 'PS-SWA', 'IAI', 'Trad.']

    pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"    

    # Reorder columns based on custom order
    pivot_table = pivot_table.reindex(columns=custom_column_order)

    styled_df = pivot_table.style.apply(bold_max, axis=1)

    return styled_df

def put_latex_tables_side_by_side(left_latex, right_latex):
    return r"\begin{tabular}{ccccccc}\hline"+left_latex+r"\\ \hline\end{tabular}\quad\begin{tabular}{ccccccc}\hline"+right_latex+r"\\ \hline\end{tabular}"

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "{SA}": r"{\rotcell{SA}}", # note that SA is a subset of SAT\_50 etc.., so it causes some issues
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                    r"\multirow[c]{12}" : r"\hline \multirow[c]{12}",
                    r"& \multirow[c]{3}" : r"\cline{2-7} & \multirow[c]{3}",
                    r"} \cline{2-7}" : "} ",
                   "\\font-weightbold": "",
                    "≪": "\ll "}

    texts_to_rotate = ["problem", "BT", "GC\_anna", "GC\_jean", "SAT\_20",  "SAT\_50",  "SAT\_100", "Met.", "GA", "Tabu", "RS", "PSO", "depth"]  # MOD: added PSO to rotate label in LaTeX output

    for item_to_rotate in texts_to_rotate:
        replacements[item_to_rotate] = r"\rotcell{"+item_to_rotate+"}"

    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string


def pivot_table_as_latex(pivot_table):
    latex_text = pivot_table.to_latex(convert_css=True)
    latex_text = fix_latex(latex_text)
    return latex_text

In [14]:
pRef_size = 10000
depths = [3, 4, 5]


usable_data = accuracy_data.copy()
usable_data = usable_data[usable_data["pRef_size"] == pRef_size] 
usable_data = usable_data[usable_data["depth"].isin(depths)]

independent_variables = ["problem", "pRef_method", "kind", "depth"]
dependent_variables = ["r_sq"]


problems = ["BT", "GC_S", "GC_L", "SAT_S", "SAT_M", "SAT_L"]

left_problems, right_problems = problems[:3], problems[3:]

def make_table_for_problems(problem_subset):
    with_right_problems = usable_data[usable_data['problem'].isin(problem_subset)]
    pivot_table = with_right_problems.pivot_table(index = ["problem", "pRef_method", "depth"], 
                                        columns = ["kind"], 
                                        values = dependent_variables[0])
    return style_pivot_table(pivot_table)



left_table = make_table_for_problems(left_problems)
right_table = make_table_for_problems(right_problems)

display(left_table)
display(right_table)

left_table_latex = pivot_table_as_latex(left_table)
right_table_latex = pivot_table_as_latex(right_table)

full_table_latex = put_latex_tables_side_by_side(left_table_latex, right_table_latex)

print("left table:")
print(left_table_latex)

print("\n\n\n\n\n\nright table")
print(right_table_latex)



# pivot_table = usable_data.pivot_table(index = ["problem", "pRef_method", "depth"], 
#                                         columns = ["kind"], 
#                                         values = dependent_variables[0])





# for problem in usable_data['problem'].unique():
#     with_right_problem = usable_data[usable_data['problem'] == problem]
#     pivot_table = with_right_problem.pivot_table(index = ["pRef_method", "depth"], 
#                                         columns = ["kind"], 
#                                         values = dependent_variables[0])

#     pivot_table = style_pivot_table(pivot_table)
#     print(fix_latex(f"{problem = }"))
#     print_pivot_table_as_latex(pivot_table)
    
    # Display the styled dataframe
    #display(styled_df)
    
    #display(pivot_table)


left table:
\begin{tabular}{lllllll}
 &  & tree & PS-SW & PS-SWA & IAI & Trad. \\
\rotcell{problem} & \rotcell{Met.} & \rotcell{depth} &  &  &  &  \\
\multirow[c]{21}{*}{\rotcell{GC\_anna}}  & \multirow[c]{3}{*}{ABC} & 3 & nan\% & nan\% & 0.0\% & 0.3\% \\
 &  & 4 & nan\% & nan\% & 0.0\% & 0.6\% \\
 &  & 5 & nan\% & nan\% & 0.0\% & 0.4\% \\
 \cline{2-7} & \multirow[c]{3}{*}{ACO} & 3 & nan\% & nan\% & -0.4\% & -0.2\% \\
 &  & 4 & nan\% & nan\% & -0.0\% & 0.1\% \\
 &  & 5 & nan\% & nan\% & 0.3\% & -1.0\% \\
 \cline{2-7} & \multirow[c]{3}{*}{BBO} & 3 & nan\% & nan\% & -0.5\% & 0.1\% \\
 &  & 4 & nan\% & nan\% & -0.5\% & -0.3\% \\
 &  & 5 & nan\% & nan\% & -0.5\% & -2.0\% \\
 \cline{2-7} & \multirow[c]{3}{*}{BRO} & 3 & -0.0\% & -0.0\% & -0.2\% & \bfseries 0.1\% \\
 &  & 4 & \bfseries -0.0\% & -2.0\% & -0.5\% & -0.6\% \\
 &  & 5 & -2.7\% & -2.2\% & \bfseries -0.1\% & -0.4\% \\
 \cline{2-7} & \multirow[c]{3}{*}{CRO} & 3 & \bfseries 0.0\% & -0.6\% & \bfseries -0.0\% & -0.9\% \\
 &  & 4 & -1.8\

In [15]:
result = []
print(accuracy_data["problem"].unique())
print(accuracy_data["pRef_method"].unique())
for method in accuracy_data["pRef_method"].unique():
    for problem in accuracy_data["problem"].unique():
        for depth in [3, 4, 5]:
            appropriate_data = filter_dataframe(accuracy_data, depth = depth, pRef_method = method, problem = problem, pRef_size=10000)
            trad_data = appropriate_data[appropriate_data["kind"] == "Trad."]["r_sq"][:100]
            own_data = appropriate_data[appropriate_data["kind"] == "PS-SW"]["r_sq"][:100]
            p_value = mannwhitneyu(x = own_data, y = trad_data, alternative="greater").pvalue
            diff = np.average(own_data)-np.average(trad_data)
            
            p_value_string = "0.001" if p_value < 0.001 else round(p_value, 3)
            res = f"{round(diff*100, 2)}%, {p_value_string}"
            if p_value < 0.05:
                res = "\\bfseries "+res
            
            
            result.append({"method":method,
                           "problem":problem,
                           "depth":depth,
                           "res":res})
            
        
improvements = pd.DataFrame(result)
display(improvements)
pivot_table = improvements.pivot_table(index=["problem", "method"],
                                       columns="depth",
                                       values=["res"],
                                       aggfunc=lambda x:x)

#pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"    

display(pivot_table)

latex_code = pivot_table.to_latex()

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "BT":"Staff R.",
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                   "\\font-weightbold": "",
                    "res": "average R^2 improvemennt between PS-SW and Trad, with p-value",
                    "≪": "\ll "}


    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string
latex_code = fix_latex(latex_code)
print(latex_code)


['GC_L' 'GC_S' 'SAT_S' 'SAT_M' 'SAT_L']
['BRO' 'CRO' 'PSO' 'ABC' 'ACO' 'BBO' 'SSO']


C:\Users\chand\AppData\Local\Temp\ipykernel_24148\572926669.py:10: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  p_value = mannwhitneyu(x = own_data, y = trad_data, alternative="greater").pvalue
c:\Users\chand\anaconda3\Lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\chand\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,method,problem,depth,res
0,BRO,GC_L,3,"-0.07%, 1.0"
1,BRO,GC_L,4,"0.56%, 0.5"
2,BRO,GC_L,5,"-2.34%, 1.0"
3,BRO,GC_S,3,"-0.77%, 1.0"
4,BRO,GC_S,4,"-1.2%, 1.0"
5,BRO,GC_S,5,"-1.59%, 1.0"
6,BRO,SAT_S,3,"-34.05%, 1.0"
7,BRO,SAT_S,4,"-42.07%, 1.0"
8,BRO,SAT_S,5,"-27.93%, 1.0"
9,BRO,SAT_M,3,"0.83%, 0.5"


res                            
depth                      3             4             5
problem method                                          
GC_L    ABC        nan%, nan     nan%, nan     nan%, nan
        ACO        nan%, nan     nan%, nan     nan%, nan
        BBO        nan%, nan     nan%, nan     nan%, nan
        BRO      -0.07%, 1.0    0.56%, 0.5   -2.34%, 1.0
        CRO       0.96%, 0.5   -0.51%, 1.0   -0.08%, 1.0
        PSO       0.34%, 0.5    1.51%, 0.5   -1.32%, 1.0
        SSO        nan%, nan     nan%, nan     nan%, nan
GC_S    ABC        nan%, nan     nan%, nan     nan%, nan
        ACO        nan%, nan     nan%, nan     nan%, nan
        BBO        nan%, nan     nan%, nan     nan%, nan
        BRO      -0.77%, 1.0    -1.2%, 1.0   -1.59%, 1.0
        CRO      -0.67%, 1.0   -0.69%, 1.0   -2.47%, 1.0
        PSO       1.01%, 0.5    0.13%, 0.5    2.74%, 0.5
        SSO        nan%, nan     nan%, nan     nan%, nan
SAT_L   ABC        nan%, nan     nan%, nan     nan%, nan
        ACO        nan%, nan     nan%, nan     nan%, nan
        BBO        nan%, nan     nan%, nan     nan%, nan
        BRO       1.47%, 0.5    3.39%, 0.5    1.81%, 0.5
        CRO       1.91%, 0.5   -1.87%, 1.0   -7.29%, 1.0
        PSO        1.0%, 0.5   -0.23%, 1.0   -1.11%, 1.0
        SSO        nan%, nan     nan%, nan     nan%, nan
SAT_M   ABC        nan%, nan     nan%, nan     nan%, nan
        ACO        nan%, nan     nan%, nan     nan%, nan
        BBO        nan%, nan     nan%, nan     nan%, nan
        BRO       0.83%, 0.5   -0.41%, 1.0   -2.56%, 1.0
        CRO       2.35%, 0.5    4.24%, 0.5    1.23%, 0.5
        PSO       0.19%, 0.5   -1.48%, 1.0   -0.95%, 1.0
        SSO        nan%, nan     nan%, nan     nan%, nan
SAT_S   ABC        nan%, nan     nan%, nan     nan%, nan
        ACO        nan%, nan     nan%, nan     nan%, nan
        BBO        nan%, nan     nan%, nan     nan%, nan
        BRO     -34.05%, 1.0  -42.07%, 1.0  -27.93%, 1.0
        CRO       3.28%, 0.5  -38.41%, 1.0  -25.46%, 1.0
        PSO      -15.4%, 1.0  -39.67%, 1.0  -16.22%, 1.0
        SSO        nan%, nan     nan%, nan     nan%, nan

\begin{tabular}{lllll}
\toprule
 &  & \multicolumn{3}{r}{average R^2 improvemennt between PS-SW and Trad, with p-value} \\
 & depth & 3 & 4 & 5 \\
problem & method &  &  &  \\
\midrule
\multirow[t]{7}{*}{GC\_anna} & ABC & nan\%, nan & nan\%, nan & nan\%, nan \\
 & ACO & nan\%, nan & nan\%, nan & nan\%, nan \\
 & BBO & nan\%, nan & nan\%, nan & nan\%, nan \\
 & BRO & -0.07\%, 1.0 & 0.56\%, 0.5 & -2.34\%, 1.0 \\
 & CRO & 0.96\%, 0.5 & -0.51\%, 1.0 & -0.08\%, 1.0 \\
 & PSO & 0.34\%, 0.5 & 1.51\%, 0.5 & -1.32\%, 1.0 \\
 & SSO & nan\%, nan & nan\%, nan & nan\%, nan \\
\cline{1-5}
\multirow[t]{7}{*}{GC\_jean} & ABC & nan\%, nan & nan\%, nan & nan\%, nan \\
 & ACO & nan\%, nan & nan\%, nan & nan\%, nan \\
 & BBO & nan\%, nan & nan\%, nan & nan\%, nan \\
 & BRO & -0.77\%, 1.0 & -1.2\%, 1.0 & -1.59\%, 1.0 \\
 & CRO & -0.67\%, 1.0 & -0.69\%, 1.0 & -2.47\%, 1.0 \\
 & PSO & 1.01\%, 0.5 & 0.13\%, 0.5 & 2.74\%, 0.5 \\
 & SSO & nan\%, nan & nan\%, nan & nan\%, nan \\
\cline{1-5}
\multirow[t]{7}{*}{SA